# Theano version of simple DNN version 5

***What this version done ***
1. Add layer dynamically to 6 layers
2. Solve dropout bug -- predict problem, prob. problem
3. momentum dynamically increase from 0.5 to 0.9 for first 20 epochs, and using 0.9 till end (or to 0.999)
4. Add max norm to each hidden layer

***Todo***
2. ~~Add other method -- nesterov_momentum, relu, tanh~~
3. debug auto adjust since we just linearly divide by max_epoch for every epoch
4. [Add keyboardinterrupt exception then save the model](http://stackoverflow.com/questions/21120947/catching-keyboardinterrupt-in-python-during-program-shutdown)
5. Add unsupervised learning method http://www.csri.utoronto.ca/~hinton/absps/googlerectified.pdf
6. Change data to filter-bank
7. ~~fix X.dot(w)+b~~
8. ~~Check dropout method problem~~
9. Add maxout network
- ~~Add regularization term to every hidden layer~~
- Add regularization to cost function
-------

***Tune variable***
1. Change hidden layer size
2. Change dropout ratio
3. Change activation function
4. [Weight initializate](http://deeplearning.net/tutorial/mlp.html#weight-initialization)
5. > We used probability of retention p = 0.8 in the input layers and 0.5 in the hidden layers.
Max-norm constraint with c = 4 was used in all the layers. A momentum of 0.95 with a
high learning rate of 0.1 was used. The learning rate was decayed as $\epsilon_0 \frac{1}{(1 + t/T)}$, from [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)

***Knowledge need to fill the gap***
1. Numpy broadcast http://wiki.scipy.org/EricsBroadcastingDoc
2. Rewrite deep equation

***Question***
1. Why dont need a bias term b in the equation or even initialize?
2. Once apply dropout, the result is strange
3. Softmax weight divide by 2 ??

***Github***
- https://gist.github.com/SnippyHolloW/8a0f820261926e2f41cc
- https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne.py
- https://github.com/benanne/Lasagne/blob/master/lasagne/nonlinearities.py
- https://github.com/nitishsrivastava/deepnet/tree/master/deepnet

***Paper***
1. [Improving neural networks by preventing
co-adaptation of feature detectors](http://arxiv.org/pdf/1207.0580.pdf)
2. [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
3. [Maxout Network](http://arxiv.org/pdf/1302.4389v4.pdf)

***References***
0. [Ipython Markdown](http://nbviewer.ipython.org/github/twistedhardware/mltutorial/blob/master/notebooks/IPython-Tutorial/2%20-%20Markdown%20%26%20LATEX.ipynb)
- Dropout, Hinton http://arxiv.org/pdf/1207.0580.pdf
- Dropout, Pratical guide http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf
- Dropout, MaxOutnetwork http://arxiv.org/pdf/1302.4389v4.pdf
- [Several Deep](http://snippyhollow.github.io/blog/2014/08/09/so-you-wanna-try-deep-learning/) -- Dropout, you will see improvement only on large-enough networks (> 1000 units / layer, > 3-4 layers **[example](https://gist.github.com/SnippyHolloW/8a0f820261926e2f41cc)**
- Using cudamat implementation deep: expert github https://github.com/nitishsrivastava/deepnet
2. Tutorial http://neuralnetworksanddeeplearning.com/chap3.html
3. http://ieeexplore.ieee.org/xpl/articleDetails.jsp?tp=&arnumber=6296526&searchWithin%3Dp_Authors%3A.QT.Hinton%2C+G..QT.%26searchWithin%3Dp_Author_Ids%3A37270925500
4. http://ieeexplore.ieee.org/xpl/articleDetails.jsp?tp=&arnumber=5704567&searchWithin%3Dp_Authors%3A.QT.Hinton%2C+G..QT.%26searchWithin%3Dp_Author_Ids%3A37270925500%26sortType%3Ddesc_p_Publication_Year
5. Kaldi, acostic preprocess http://kaldi.sourceforge.net/

In [16]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
import numpy as np

from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn import grid_search as gs
from sklearn.metrics import accuracy_score
# from sklearn.base import BaseEstimator

from time import time

## Data preprocess

In [17]:
### DataProcessing 

'''
[ INPUT  ] : model { wav, NFCC, ... }
[ OUTPUT ] : (Training_data, Validation_data, Testing_Data)

- Training_data : ( x, y, id )
- Validation_data : ( x, y, id )
- Testing_Data : ( x, id )

-- x format : 
---- NFCC : [ 39-vector ]'

-- y format :
---- Now we use 39-phonemes for all : [ 0 0 0 ... 1 .... 0 0 0 ]  as a number of 39
'''

def Dataset ( model, dratio ) :
    trains = {}
    tests_data = {}
    phones_mapping = {} # {48} to realNumber
    result_mapping = {} # {48} to {39}
    
    training_inputs = []
    training_result = []
    if model == "mfcc":
        # TRAINING X(INPUT)
        with open('./MLDS_HW1_RELEASE_v1/mfcc/train.ark') as f:
            for lines in f :
                frames = lines.split(' ')
                frame2float = [ float(x) for x in frames[1:] ]
                # trains[frames[0]] = frames[1:]
                trains[frames[0]] = frame2float

        # MAPPING 48 to number ( we map 48 to 39 later )
        with open('./MLDS_HW1_RELEASE_v1/phones/48_39.map') as f:    
            i = 0
            for lines in f :
                phones = lines.split('\t')
                phones_mapping[phones[0]] = i
                i += 1
        
        with open('./MLDS_HW1_RELEASE_v1/phones/48_39.map') as f:    
            for lines in f :
                phones = lines.split('\t')
                result_mapping[ phones_mapping[phones[0]] ] = phones[1]
        
        # TRAINING Y(OUTPUT)
        with open('./MLDS_HW1_RELEASE_v1/label/train.lab') as f:
            for lines in f :
                labels = lines.split(',')
                # trains[labels[0]].append(labels[1])
                training_inputs.append( np.reshape(trains.get(labels[0]), (39, 1) ) )
                training_result.append( vectorized_result(phones_mapping[labels[1].rstrip('\n')] ) )
        
        # 10% for validation
        dataSize_weUse = len(training_inputs) * dratio
        trainingRationTEST = int( dataSize_weUse * 0.9)
        trainingRationVARI = int( dataSize_weUse * 0.1)
        trainingRationVARIend = trainingRationVARI + trainingRationTEST
        print "slide ratio : " , trainingRationTEST
        if dratio != 1:
            training_data = zip(training_inputs[0:trainingRationTEST], training_result[0:trainingRationTEST])
            validation_data = zip(training_inputs[trainingRationTEST+1:trainingRationVARIend], training_result[trainingRationTEST+1:trainingRationVARIend])
        else:
        
            X_train, X_test, y_train, y_test = cv.train_test_split(training_inputs, training_result, test_size=0.1)
            y_train = np.array(y_train)
            y_train = ((y_train.flatten())).reshape(len(y_train.flatten())/48,48).astype(np.float32)
            X_train = np.array(X_train)
            X_train = ((X_train.flatten())).reshape(len(X_train.flatten())/39,39).astype(np.float32)
            y_test = np.array(y_test)
            y_test = ((y_test.flatten())).reshape(len(y_test.flatten())/48,48).astype(np.float32)
            X_test = np.array(X_test)
            X_test = ((X_test.flatten())).reshape(len(X_test.flatten())/39,39).astype(np.float32)
            
            
#             training_data = zip(training_inputs[0:trainingRationTEST], training_result[0:trainingRationTEST])
#             validation_data = zip(training_inputs[trainingRationTEST+1:], training_result[trainingRationTEST+1:])
            
#         print "Size of Training Data", len(training_data)
#         print "Size of Validation Data", len(validation_data)
        
        # Testing data
        with open('./MLDS_HW1_RELEASE_v1/mfcc/test.ark') as f:
            # i = 0
            for lines in f :
                frames = lines.split(' ')
                tests_data[frames[0]] = np.reshape([ float(x) for x in frames[1:] ], (39, 1) )
                # if i < 20:
                #     i += 1
                # else:
                #    break
        
#         tests_data = np.array(tests_data.values())
#         tests_data = np.array(((tests_data.flatten()))).reshape(len(tests_data.flatten())/39,39).astype(np.float32)
                
    else:
        print "Not implement yet"
        
#     return ( training_data, validation_data, tests_data, result_mapping )
    return (X_train, X_test, y_train, y_test, tests_data, result_mapping)
                                       
def vectorized_result ( j ) :
    e = np.zeros((48, 1))
    e[j] = 1.0
    return e

## Model part


In [65]:
srng = RandomStreams()

# translate data to theano data type
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

# initialize weight by random
def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

def tanh(X):
    return (1 + T.tanh(X / 2)) / 2

# rectified linear unit
def relu(X):
    # return T.maximum(X, 0.)
    return (X + abs(X)) / 2.

def sigmoid(X):
    return 1.0/(1.0+ T.exp(-X))

# softmax
def softmax(X):
    e_x = T.exp(X - X.max(axis=1).dimshuffle(0, 'x'))
    return e_x / e_x.sum(axis=1).dimshuffle(0, 'x')

def build_shared_zeros(shape):
    """ Builds a theano shared variable filled with a zeros numpy array """
    return theano.shared(value=np.zeros(shape, dtype=theano.config.floatX),
             borrow=True)

# method provided by Hinton
def RMSprop(cost, params, lr=0.001, rho=0.9, epsilon=1e-6):
    grads = T.grad(cost=cost, wrt=params)
    updates = []
    for p, g in zip(params, grads):
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        gradient_scaling = T.sqrt(acc_new + epsilon)
        g = g / gradient_scaling
        updates.append((acc, acc_new))
        updates.append((p, p - lr * g))
    return updates

# momentum method
def momentum(loss, all_params, update_momentum, update_learning_rate, max_norm):
    all_grads = theano.grad(loss, all_params)
    updates = []

    for param_i, grad_i in zip(all_params, all_grads):
        mparam_i = theano.shared(np.zeros(param_i.get_value().shape,
                                          dtype=theano.config.floatX),
                                 broadcastable=param_i.broadcastable)
        v = update_momentum * mparam_i - update_learning_rate * grad_i
        updates.append((mparam_i, v))
        
        if max_norm != None:
            W = param_i + v
            col_norms = W.norm(2, axis=0)
            desired_norms = T.clip(col_norms, 0, max_norm)
            updates.append( ( param_i , (W * (desired_norms / (1e-6 + col_norms))) ) )
        else:
            updates.append((param_i, param_i + v))

    return updates

def multinominal_cross_entropy(z, X):
    
    L = - T.sum(X * T.log(z) + (1 - X) * T.log(1 - z), axis=1)
    loss = T.sum(L) / X.shape[0]
    
    return loss

def negative_log_likelihood_mean(z, X):
    return -T.mean(T.log(z)[T.arange(X.shape[0]), X])

def negative_log_likelihood_sum(z, X):
    return -T.sum(T.log(z)[T.arange(X.shape[0]), X])


# dropout
# https://github.com/mdenil/dropout/issues/6
def dropout(X, p=0.):
    if p > 0. and p < 1.:
        retain_prob = 1 - p
        X *= srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
    return X


def model(X, para, p_drop_input, p_drop_hidden, relu_bool, train):
    ## assume 6 hidden  layers only
    ## Softmax weight divide by 2??
    for i in range (len(para)/2):
        
        if i is 0:
            ######## Input layer
            if train:
                X = dropout(X, p_drop_input)
            else:
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
                
            if relu_bool:
                h = relu(T.dot(X, para[i*2]) +para[i+1])
            else:
                h = sigmoid(T.dot(X, para[i*2]) + para[i+1])

        elif i is 1:
            ######## layer 1
            if train:
                h = dropout(h, p_drop_hidden[i-1])
            else:
                # h *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 4, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h2 = relu(T.dot(h, para[i*2] )+ para[i*2+1])
            else: 
                h2 = sigmoid(T.dot(h, para[i*2] ) +para[i*2+1])
        
        elif i is 2:
            ######## layer 2
            if train:
                h2 = dropout(h2, p_drop_hidden[i-1])
            else:
                # h2 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 6, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h2, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h3 = relu(T.dot(h2, para[i*2] )+ para[i*2+1])
            else:
                h3 = sigmoid(T.dot(h2, para[i*2] ) +para[i*2+1])
                
            
        elif i is 3:
            ######## layer 3
            if train:
                h3 = dropout(h3, p_drop_hidden[i-1])
            else:
                # h3 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 8, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h3, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h4 = relu(T.dot(h3, para[i*2] )+ para[i*2+1])
            else:
                h4 = sigmoid(T.dot(h3, para[i*2] ) +para[i*2+1])
                
                
        elif i is 4:
            ######## layer 4
            if train:
                h4 = dropout(h4, p_drop_hidden[i-1])
            else:
                # h4 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 10, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h4, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h5 = relu(T.dot(h4, para[i*2] )+ para[i*2+1])
            else:
                h5 = sigmoid(T.dot(h4, para[i*2] ) +para[i*2+1])
                
                
        elif i is 5:
            ######## layer 5
            if train:
                h5 = dropout(h5, p_drop_hidden[i-1])
            else:
                # h5 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 12, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h5, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h6 = relu(T.dot(h5, para[i*2] )+ para[i*2+1])
            else:
                h6 = sigmoid(T.dot(h5, para[i*2] ) +para[i*2+1])
                
        elif i is 6:
            ######## layer 6
            if train:
                h6 = dropout(h6, p_drop_hidden[i-1])
            else:
                # h6 *= (1-p_drop_hidden[i-1])
                para[i*2].set_value(para[i*2].get_value()*(1-p_drop_hidden[i-1]))
            
            if (i*2+2) is len(para): ## weight at least have 12, w_h_1, b_1, w_o, b_o
                py_xx = softmax(T.dot(h6, para[i*2])+ para[i*2+1])
                break
            
            if relu_bool:
                h7 = relu(T.dot(h6, para[i*2] )+ para[i*2+1])
            else:
                h7 = sigmoid(T.dot(h6, para[i*2] ) +para[i*2+1])
                
    return py_xx
    
    

# def model(X, w_h_1, b_1, 
#           w_h_2, b_2, 
#           w_h_3, b_3, 
#           w_h_4, b_4, 
#           w_h_5, b_5, 
#           w_h_6, b_6, 
#           w_o, b_o, 
#           p_drop_input, 
#           p_drop_hidden_1, p_drop_hidden_2, p_drop_hidden_3,
#           p_drop_hidden_4, p_drop_hidden_5, p_drop_hidden_6,
#           relu, train):
    
#     if train:
#         X = dropout(X, p_drop_input)
    
#     ######## layer 1
#     if !relu:
#         h = sigmoid(T.dot(X, w_h_1) +b_1)
#     else:
#         h = relu(T.dot(X, w_h_1) +b_1)

#     if train:
#         h = dropout(h, p_drop_hidden_1)
#     else:
#         h *= (1-p_drop_hidden_1)
    
#     ######## layer 2
#     if !relu:
#         h2 = sigmoid(T.dot(h, w_h_2 ) +b_2)
#     else:
#         h2 = relu(T.dot(h, w_h_2 )+b_2)
    
#     if train:
#         h2 = dropout(h2, p_drop_hidden_2)
#     else:
#         h2 *= (1-p_drop_hidden_2)
    
#     ######## layer 3
#     if !relu:
#         h3 = sigmoid(T.dot(h2, w_h_3) +b_3)
#     else:
#         h3 = relu(T.dot(h2, w_h_3)+b_3)
    
#     if train:
#         h3 = dropout(h3, p_drop_hidden_3)
#     else:
#         h3 *= (1-p_drop_hidden_3)
    
#     ######## layer 4
        
#     if !relu:
#         h4 = sigmoid(T.dot(h3, w_h_4) +b_4)
#     else:
#         h4 = relu(T.dot(h3, w_h_4)+b_4)

#     if train:
#         h4 = dropout(h4, p_drop_hidden_4)
#     else:
#         h4 *= (1-p_drop_hidden_4)
    
#     ######## layer 5
#     if !relu:
#         h5 = sigmoid(T.dot(h4, w_h_5) +b_5)
#     else:
#         h5 = relu(T.dot(h4, w_h_5)+b_5)

#     if train:
#         h5 = dropout(h5, p_drop_hidden_5)
#     else:
#         h5 *= (1-p_drop_hidden_5)
        
#     ######## layer 6
#     if !relu:
#         h6 = sigmoid(T.dot(h5, w_h_6) +b_6)
#     else:
#         h6 = relu(T.dot(h5, w_h_6)+b_6)
    
#     if train:
#         h6 = dropout(h6, p_drop_hidden_6)
#     else:
#         h6 *= (1-p_drop_hidden_6)

#     ######## output layer
#     # when dropout, softmax weight need to devide by 2 ?
#     py_x = softmax(T.dot(h6, w_o) +b_o) 
    
#     return py_x


# def model(X, w_h, w_o):
#     h = sigmoid(T.dot(X,w_h))
#     py_x = softmax(T.dot(h,w_o))
    
#     return py_x

In [66]:
class DNN():    
    
    def __init__(self, input_shape, activation, hidden_layer,
                 batch, max_epochs, eval_size, output_num_units,
                 drop_input, drop_hidden,
                 patience, up_learning_rate, up_momentum, max_norm):
        
        self.input_shape = input_shape
        self.hidden_layer = hidden_layer
#         self.hidden_layer_1 = hidden_layer_1
#         self.hidden_layer_2 = hidden_layer_2
#         self.hidden_layer_3 = hidden_layer_3
        
        if activation == "relu":
            self.activation = True
        else:
            self.activation = False
        
        self.batch = batch
        self.max_epochs = max_epochs
        self.eval_size = eval_size
        self.output_num_units = output_num_units
        
        self.up_learning_rate = up_learning_rate
        self.up_momentum = up_momentum
        
        self.drop_input = drop_input
        self.drop_hidden = drop_hidden
#         self.drop_hidden_1 = drop_hidden_1
#         self.drop_hidden_2 = drop_hidden_2
#         self.drop_hidden_3 = drop_hidden_3
        
        self.patience = patience
        self.best_valid = -np.inf
        self.best_valid_epoch = 0
        self.best_weights = None
        
        self.max_norm = max_norm
        
        self.train_history_ = []

        self.params = []
        for i in range(len(hidden_layer)):
            if i is 0:
                w_h_1 = init_weights((self.input_shape[1], self.hidden_layer[i]))
                b_1 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_1)
                self.params.append(b_1)
            elif i is 1:
                w_h_2 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_2 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_2)
                self.params.append(b_2)
            elif i is 2:
                w_h_3 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_3 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_3)
                self.params.append(b_3)
            elif i is 3:
                w_h_4 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_4 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_4)
                self.params.append(b_4)
            elif i is 4:
                w_h_5 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_5 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_5)
                self.params.append(b_5)
            elif i is 5:
                w_h_6 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_6 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_6)
                self.params.append(b_6)
            elif i is 6:
                w_h_7 = init_weights((self.hidden_layer[i-1], self.hidden_layer[i]))
                b_7 = build_shared_zeros(self.hidden_layer[i])
                self.params.append(w_h_7)
                self.params.append(b_7)
                
            if (i+1) is len(hidden_layer):
                w_o = init_weights((self.hidden_layer[i], self.output_num_units))
                b_o = build_shared_zeros(self.output_num_units)
                self.params.append(w_o)
                self.params.append(b_o)
                break

            
#         w_h_1 = init_weights((self.input_shape[1], self.hidden_layer_1))
#         w_h_2 = init_weights((self.hidden_layer_1, self.hidden_layer_2))
#         w_h_3 = init_weights((self.hidden_layer_2, self.hidden_layer_3))
#         w_h_4 = init_weights((self.hidden_layer_3, self.hidden_layer_4))
#         w_h_5 = init_weights((self.hidden_layer_4, self.hidden_layer_5))
#         w_h_6 = init_weights((self.hidden_layer_5, self.hidden_layer_6))
#         w_o = init_weights((self.hidden_layer_6, self.output_num_units))
        
#         b_1 = build_shared_zeros(self.hidden_layer_1)
#         b_2 = build_shared_zeros(self.hidden_layer_2)
#         b_3 = build_shared_zeros(self.hidden_layer_3)
#         b_4 = build_shared_zeros(self.hidden_layer_4)
#         b_5 = build_shared_zeros(self.hidden_layer_5)
#         b_6 = build_shared_zeros(self.hidden_layer_6)
#         b_o = build_shared_zeros(self.output_num_units)
        
        self.update_learning_rate= theano.shared(floatX( up_learning_rate['start'] ))
        self.lr = np.linspace(up_learning_rate['start'], up_learning_rate['stop'], self.max_epochs)
        # self.lr = np.linspace(up_learning_rate['start'], up_learning_rate['stop'], 20)
        
        self.update_momentum= theano.shared(floatX( up_momentum['start'] ))
        # self.mm = np.linspace(up_momentum['start'], up_momentum['stop'], self.max_epochs)
        self.mm = np.linspace(up_momentum['start'], up_momentum['stop'], up_momentum['epoch'])
    
        X = T.dmatrix()
        Y = T.dmatrix()

        # Construct Theano expression graph
#         py_x_drop = model(X, 
#                      w_h_1,b_1,
#                      w_h_2,b_2,
#                      w_h_3,b_3, 
#                      w_h_4,b_4, 
#                      w_h_5,b_5, 
#                      w_h_6,b_6, 
#                      w_o, b_o,
#                      self.drop_input, 
#                      self.drop_hidden_1, self.drop_hidden_2, self.drop_hidden_3, 
#                      self.drop_hidden_4, self.drop_hidden_5, self.drop_hidden_6,
#                      self.activation, True)
        
#         py_x = model(X, 
#                      w_h_1,b_1,
#                      w_h_2,b_2,
#                      w_h_3,b_3, 
#                      w_h_4,b_4, 
#                      w_h_5,b_5, 
#                      w_h_6,b_6, 
#                      w_o, b_o,
#                      self.drop_input, 
#                      self.drop_hidden_1, self.drop_hidden_2, self.drop_hidden_3, 
#                      self.drop_hidden_4, self.drop_hidden_5, self.drop_hidden_6,
#                      self.activation, False)
        
        py_x_drop = model(X, self.params, 
                     self.drop_input, self.drop_hidden, self.activation, True)
    
        py_x = model(X, self.params, 
                     self.drop_input, self.drop_hidden, self.activation, False)

        y_x = T.argmax(py_x, axis=1)

        # cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
        cost = multinominal_cross_entropy(py_x_drop, Y) 
        
#         self.params = [ w_h_1, b_1, 
#                        w_h_2, b_2, 
#                        w_h_3, b_3, 
#                        w_h_4, b_4,
#                        w_h_5, b_5,
#                        w_h_6, b_6,
#                        w_o, b_o]
        
        updates = momentum(cost, self.params, self.update_momentum, self.update_learning_rate, self.max_norm)

        # Compile expressions to functions
        self.train = theano.function(inputs=[X, Y], outputs=[cost], 
                                updates=updates, allow_input_downcast=True, name = "train")
        self.predict = theano.function(inputs=[X], outputs=y_x, allow_input_downcast=True, name = "predict")

        
    def fit(self, x, y):
        X_train, X_test, y_train, y_test = cv.train_test_split(x, y, test_size= self.eval_size)
        yy = np.array(map((lambda x: np.argmax(x)), y_test))
        
        if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
        self.train.maker.fgraph.toposort()]):
            print 'Used the cpu'
        elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
                  self.train.maker.fgraph.toposort()]):
            print 'Used the gpu'
        else:
            print 'ERROR, not able to tell if theano used the cpu or the gpu'
            print self.train.maker.fgraph.toposort()
            
        print " "
        print "start training!!!!"
        print " "
            
        epochs = 0
        for i in range(self.max_epochs):
            epochs +=1
            t0 = time()
            
            # To do: modify a function of mini batch and using random
            for start, end in zip(range(0, len(X_train), self.batch), range(self.batch, len(X_train), self.batch)):
                err = self.train(X_train[start:end], y_train[start:end])
            
            score = accuracy_score(yy, self.predict(X_test))
            self.train_history_.append({"epoch":epochs, "err": err, "score":score})
            
            print 'epoch {0} : err = {1}, score = {2}, time ={3} s'.format(epochs, err, score, time() - t0)
            
            if np.isnan(err):
                break
            
            if epochs != 1:
                new_lr = floatX(self.lr[epochs - 1])
                self.update_learning_rate.set_value(new_lr)
                if epochs <= self.up_momentum['epoch']:
                    new_mm = floatX(self.mm[epochs - 1])
                    self.update_momentum.set_value(new_mm)
                    print "momentum update: ", new_mm
                
            current_score = self.train_history_[-1]['score']
            current_epoch = self.train_history_[-1]['epoch']
            if current_score > self.best_valid:
                self.best_valid = current_score
                self.best_valid_epoch = current_epoch
                self.best_weights = [w.get_value() for w in self.params]
            elif self.best_valid_epoch + self.patience <= current_epoch:
                print ""
                print "Early stopping."
                print self.best_valid_epoch,self.best_valid
                print "Best valid score {:.6f} at epoch {}.".format(self.best_valid, self.best_valid_epoch)
                
                for qq in range (len(self.params)):
                    self.params[qq].set_value( self.best_weights[qq] )
                break


    def prediction(self, x):
        return self.predict(x)

    def outputCSV(self, wfilename, test_data, mapd): # read dictionary for id 
        test_results = []
                
        for xid, xdata in test_data.iteritems():
            test_results.append( (xid, self.predict(xdata.T)) ) 
            
        f = open(wfilename, 'w+')
        f.write("Id,Prediction\n")
        for xid, y in test_results:
            f.write("{0},{1}".format(xid, mapd[y[0]]))
        f.close()
        print "MISSION COMPLETE"
    
    

# Start training!

 ## Data reading

In [6]:
### Data Processing 

data_ratio = 1 # the input we use ( to put more efford on improve parameter)

X_train, X_test, y_train, y_test, test_data, result_mapping = Dataset("mfcc", data_ratio)


slide ratio :  1012340


In [23]:
(np.array(y_train)).shape
# y_train = np.array(y_train)
# ((y_train.flatten())).reshape(len(y_train.flatten())/48,48).astype(np.float32)
# y_train[0].flatten()

(1012340, 48)

In [ ]:
(np.array(X_train)).shape
# X_train = np.array(X_train)
# X_train[0].shape
# (X_train.flatten()).reshape(1012340,39)[0]

In [ ]:
(np.array(test_data)).shape
# len(np.array(test_data.values()[0]))
# np.array(test_data.values()).flatten()

# test_data = np.array(test_data.values())
# np.array(((test_data.flatten()))).reshape(len(test_data.flatten())/39,39).shape


## Train model!

In [69]:
net5 = DNN(
    input_shape=(128,39), 
    hidden_layer=[300, 300, 300, 300], # maximum size to 6 layer only
    drop_input=0, # usually is 0.2
    drop_hidden=[0.2,0.2,0.2,0.2], # maximum size to 6 layer only, usually use 0.5 
    activation = "relu", # relu, sigmoid
    batch=128, 
    max_epochs=600, 
    eval_size=0.1, 
    output_num_units=48, 
    up_learning_rate = {'start':0.1, 'stop':0.0001}, 
    up_momentum = {'start':0.8, 'stop':0.9, 'epoch':40}, # only 40 epochs, and after 40 epochs, use 0.9
    patience=100,
    max_norm = 4 # need to tune, maybe start from 3 or 4
)


In [12]:
# net5 = DNN(
#     input_shape=(128,39), 
#     hidden_layer_1=256, 
#     hidden_layer_2=256, 
#     hidden_layer_3=256, 
#     hidden_layer_4=256, 
#     hidden_layer_5=256, 
#     hidden_layer_6=256, 
#     activation = "sigmoid",
#     drop_input=0.2, 
#     drop_hidden_1=0.5,
#     drop_hidden_2=0.5,
#     drop_hidden_3=0.5,
#     drop_hidden_4=0.5,
#     drop_hidden_5=0.5,
#     drop_hidden_6=0.5,
#     batch=128, 
#     max_epochs=1000, 
#     eval_size=0.1, 
#     output_num_units=48, 
#     up_learning_rate = {'start':0.03, 'stop':0.0001}, 
#     up_momentum = {'start':0.9, 'stop':0.9},
#     patience=100
# )

In [ ]:
%time net5.fit(X_train, y_train)

In [ ]:
net5.train_history_

## Save model!!

In [52]:
import sys
sys.setrecursionlimit(10000)

import cPickle as pickle
with open('net5.pickle', 'wb') as f:
    pickle.dump(net5, f, -1)


## Load model

In [ ]:
import cPickle as pickle

with open('./net5.pickle', 'rb') as f:
    net5 = pickle.load(f)


## validation result

In [53]:
y_pred = net5.prediction(X_test)

yyy = np.array(map((lambda x: np.argmax(x)), y_test))

print metrics.classification_report((yyy), (y_pred))
print accuracy_score(yyy, y_pred)


             precision    recall  f1-score   support

          0       0.51      0.58      0.54      2472
          1       0.56      0.63      0.59      2808
          2       0.35      0.35      0.35      1751
          3       0.57      0.49      0.53      2249
          4       0.47      0.38      0.42      1097
          5       0.39      0.29      0.34      1873
          6       0.62      0.59      0.61      2767
          7       0.64      0.54      0.58       986
          8       0.47      0.42      0.44       742
          9       0.69      0.70      0.69      6786
         10       0.49      0.36      0.41      1169
         11       0.55      0.41      0.47      1408
         12       0.51      0.60      0.55       846
         13       0.44      0.40      0.42      2678
         14       0.47      0.38      0.42       972
         15       0.34      0.16      0.21       557
         16       0.54      0.26      0.35       411
         17       0.70      0.66      0.68   

# write predict file

In [54]:
net5.outputCSV("net5_predict.csv", test_data, result_mapping)

MISSION COMPLETE
